# **Library and stuff**

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
#!pip install cv2
import cv2 # read video file
from skimage.transform import resize # resizing images
import h5py
from  google.colab import drive
#drive.mount('/content/drive')
!free -mh

              total        used        free      shared  buff/cache   available
Mem:            12G        631M         10G        1.1M        2.0G         11G
Swap:            0B          0B          0B


In [ ]:
path = '/content/drive/'

# **Process video**

##**Take 30 frames and turn into numpy array**

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/database/HockeyFights/fight/fi1_xvid.avi')
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT); print(f"Total frame: {total_frame}")
fps = cap.get(cv2.CAP_PROP_FPS); print(f"FPS: {fps}, type: {type(fps)}")

Total frame: 41.0
FPS: 25.0, type: <class 'float'>


In [ ]:
def slice_video_1sec(file_dir, save_dir):
    """This function slice video files in file_dir directory by 1 second(30 frames).
    Sliced video files are saved in save_dir directory.
    :: file_dir :: Where original video files were saved.
    :: save_dir :: Where the sliced video files will be saved in."""
    
    file_list = os.listdir(file_dir) # Original video file list
    countt = 0
    video_count = 0;
    for file in tqdm(file_list):
        file_path = os.path.join(file_dir, file) # full path or original video file
        
        video_name = file.split('.')[0]
        cap = cv2.VideoCapture(file_path)
       
        total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT) # Total frame of Video File
        fps = 30.0#cap.get(cv2.CAP_PROP_FPS)
        W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        sliceno = 1
        frame_count = 0
        frame_arr = []
        while cap.isOpened():
            grabbed, frame = cap.read()
            
            if not grabbed : 
                #print('cap is not grabbed. Processing ends.')
                break
        
            if int(int(total_frame)-int(frame_count)) == int(total_frame) % 30:
                #print('The rest of video file is too small. Processing ends.')
                #print(int(int(total_frame)-int(frame_count)))
                break
            
            frame_arr.append(frame)
            frame_count += 1
            if len(frame_arr) >= 30:
                #print("save")
                save_path = os.path.join(save_dir, video_name  +'_' + str(sliceno) + '.mp4' )
                fourcc = cv2.VideoWriter_fourcc(*'DIVX')
                writer = cv2.VideoWriter(save_path, fourcc, fps, (W, H), True)
                for i in range(len(frame_arr)):
                    writer.write(frame_arr[i])
                writer.release()
                sliceno += 1
                frame_arr = []
                video_count+=1
        countt += 1
        cap.release()
    print(video_count)

In [ ]:
file_dir_Violence = '/content/drive/MyDrive/database/Real Life Violence Dataset/Violence/'
save_dir_Violence = '/content/drive/MyDrive/database/Violence_2/'
slice_video_1sec(file_dir_Violence, save_dir_Violence)

In [ ]:
file_dir_Violence = '/content/drive/MyDrive/database/Real Life Violence Dataset/NonViolence/'
save_dir_Violence = '/content/drive/MyDrive/database/NonViolence_2/'
slice_video_1sec(file_dir_Violence, save_dir_Violence)

##**Crop image and turn into numpy array**

In [2]:
data_full = np.zeros((400, 30, 160, 160, 3), dtype = 'float16')
label_full = [1]*200 + [0]*200

In [3]:
def make_small_data(file_dir, j_):
    file_list = os.listdir(file_dir)[600:800]
    l = len(file_list)
    j = j_
    for file_ in tqdm(file_list):
        frames = np.zeros((30, 160, 160, 3), dtype = 'float16')
        i = 0
        cap = cv2.VideoCapture(os.path.join(file_dir, file_))
        if cap.isOpened():
            grabbed, frame = cap.read()
        else:
            grabbed = False
            continue
        frm = resize(frame,(160, 160, 3))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frames[i][:] = frm
        i += 1
        while i < 30:
            grabbed, frame = cap.read()
            frm = resize(frame,(160, 160, 3))
            frm = np.expand_dims(frm,axis=0)
            if(np.max(frm)>1):
                frm = frm/255.0
            frames[i][:] = frm
            i += 1
        data_full[j][:] = frames 
        j += 1
    return j

In [4]:
def make_data(file_dir_Violence1, file_dir_Violence2, file_dir_NonViolence1, file_dir_NonViolence2):
    """Transform your video files(each file have 30 frames) to (30, 160, 160, 3) numpy array.
    And append them to video_array which is shaped (1, 30, 160, 160, 3)"""
    j = 0
    new_j = make_small_data(file_dir_Violence1,j)
    j = new_j
    #new_j = make_small_data(file_dir_Violence2,j)
    #j = new_j
    print(f"all vio:{j}")
    new_j = make_small_data(file_dir_NonViolence1,j)
    j = new_j
    #new_j = make_small_data(file_dir_NonViolence2,j)
    #j = new_j
    print(f"all vid:{j}")
    #return video_arrays, len(os.listdir(file_dir_Violence1) + os.listdir(file_dir_Violence2)), len(os.listdir(file_dir_NonViolence1) + os.listdir(file_dir_NonViolence2))

##**Make label**

In [ ]:
def make_label(): #video_arrays, l_Violence, l_NonViolence
    """This function make label of video array.
    :: video_arrays :: (None, 30, 160, 160, 3) numpy array.
    :: violence :: True = [0, 1]. False = [1, 0]"""
    #label_Violence = np.array([0, 1])
    #label_NonViolence = np.array([1, 0])
    
    #label = np.zeros((video_arrays.shape[0], 2), dtype = 'float16')
    #label = []
    i = 0
    label_full[:200][:] = 1
    label_full[200:][:] = 0
    #label[:l_Violence][:] = label_Violence #> ([0, 1])
    #label[l_Violence:][:] = label_NonViolence #> ([1, 0])
        
    #return label #> (# of video files, 2)

##**Create dateset**

In [ ]:
def create_dataset(save_filepath, file_dir_Violence,file_dir_Violence2, file_dir_NonViolence,file_dir_NonViolence2):
    """Create shuffled full datasets formatted .h5
    :: file_dir_Violence :: The folder that violence video files were saved.
    :: file_dir_NonViolence :: The folder that non-violence video files were saved.
    :: filename :: .h5 formatted dataset file will be saved."""
    data_full, l_Violence, l_NonViolence = make_data(file_dir_Violence,file_dir_Violence2, file_dir_NonViolence,file_dir_NonViolence2)
        
    label_full = make_label(data_full, l_Violence, l_NonViolence)
    #data_full,label_full = X_y_shuffle(data_full,label_full)
    #i = 2
    #with h5py.File(os.path.join(save_filepath,"dataset_" + str(i) + ".h5"), 'w') as f:
    #        f.create_dataset('data_full', data = data_full[230*i:230*(i+1)])
    #        f.create_dataset('label_full', data = label_full[230*i:230*(i+1)])
    #i = 5
    #with h5py.File(os.path.join(save_filepath,"dataset_" + str(i) + ".h5"), 'w') as f:
    #        f.create_dataset('data_full', data = data_full[230*i:230*(i+1)])
    #        f.create_dataset('label_full', data = label_full[230*i:230*(i+1)])
    #for i in range(7):
    #    with h5py.File(os.path.join(save_filepath,"dataset_" + str(i) + ".h5"), 'w') as f:
    #        f.create_dataset('data_full', data = data_full[230*i:230*(i+1)])
    #        f.create_dataset('label_full', data = label_full[230*i:230*(i+1)])
    with h5py.File(os.path.join(save_filepath,"dataset" + ".h5"), 'w') as f:
        f.create_dataset('data_full', data = data_full)
        f.create_dataset('label_full', data = label_full)

In [5]:
filename = '/content/drive/MyDrive/database/' # file directory & name where you save your datasets
file_dir_Violence = '/content/drive/MyDrive/database/Real Life Violence Dataset/Violence/'
file_dir_Violence2 = ''
file_dir_NonViolence = '/content/drive/MyDrive/database/Real Life Violence Dataset/NonViolence/'
file_dir_NonViolence2 = ''

In [6]:
make_data(file_dir_Violence,file_dir_Violence2, file_dir_NonViolence,file_dir_NonViolence2)

100%|██████████| 200/200 [05:19<00:00,  1.60s/it]


all vio:200


100%|██████████| 200/200 [02:20<00:00,  1.43it/s]

all vid:400


In [ ]:
label_full

In [7]:
with h5py.File(os.path.join(filename,"val2_dataset" + ".h5"), 'w') as f:
    f.create_dataset('data_full', data = data_full)
    f.create_dataset('label_full', data = label_full)

In [ ]:
create_dataset(filename, file_dir_Violence,file_dir_Violence2, file_dir_NonViolence,file_dir_NonViolence2)

In [ ]:
! du -sh /content/drive/MyDrive/database/*

#**Shuffle data**

In [ ]:
def X_y_shuffle(X, y):
    """This function shuffle two numpy arrays(data, label) at once with same order.
    :: X :: Video frame img features numpy array.
    :: y :: The label(target) of data. That shows violence True or False. """
    
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    return X_shuffled, y_shuffled

In [ ]:
file0 = h5py.File('/content/drive/MyDrive/database/dataset_3.h5', 'r')
file7 = h5py.File('/content/drive/MyDrive/database/dataset_4.h5', 'r')
data_full0 = file0['data_full'][:]
data_full7 = file7['data_full'][:]
label_full0 = file0['label_full'][:]
label_full7 = file7['label_full'][:]
print(len(label_full0))
print(len(label_full7))
print(label_full0[0].shape)

230
230
(2,)


In [ ]:
l = len(label_full0) + len(label_full7)
data_full = np.zeros((l, 30, 160, 160, 3), dtype = 'float16')
label_full = np.zeros((l, 2), dtype = 'float16')
for i in range(len(label_full0)):
    data_full[i][:] = data_full0[i]
    label_full[i][:] = label_full0[i]
for i in range(len(label_full7)):
    data_full[len(label_full0) + i][:] = data_full7[i]
    label_full[len(label_full0) + i][:] = label_full7[i]
print(data_full.shape)

(460, 30, 160, 160, 3)


In [ ]:
print(label_full[270])

[1. 0.]


In [ ]:
data_full, label_full = X_y_shuffle(data_full, label_full)

In [ ]:
filename = '/content/drive/MyDrive/database/new_shuffled_3.h5'
with h5py.File(filename, 'w') as f:
    f.create_dataset('data_full', data = data_full[:230])
    f.create_dataset('label_full', data = label_full[:230])
    filename = '/content/drive/MyDrive/database/new_shuffled_4.h5'
with h5py.File(filename, 'w') as f:
    f.create_dataset('data_full', data = data_full[230:])
    f.create_dataset('label_full', data = label_full[230:])

#**Cut the dataset down into 1Gb of size**

In [ ]:
#file0 = h5py.File('/content/drive/MyDrive/database/shuffled_2.h5', 'r')
#print(file0.keys())
#data_full0 = file0['data_full'][:]
#label_full0 = file0['label_full'][:]
#filename = '/content/drive/MyDrive/database/new_shuffled_4.h5'
#with h5py.File(filename, 'w') as f:
#    f.create_dataset('data_full', data = data_full0[:230])
#    f.create_dataset('label_full', data = label_full0[:230])
#    filename = '/content/drive/MyDrive/database/new_shuffled_5.h5'
#with h5py.File(filename, 'w') as f:
#    f.create_dataset('data_full', data = data_full0[230:])
#    f.create_dataset('label_full', data = label_full0[230:])

<KeysViewHDF5 ['data_full']>
